In [5]:
# A program that scrapes tables on websites.

In [21]:
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium as selenium


ModuleNotFoundError: No module named 'selenium'

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
driver = webdriver.Chrome()
url = 'https://rotogrinders.com/projected-stats/nfl-qb?site=fanduel'
driver.get(url)
table =WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.XPATH,"//*[@id='proj-stats']")))

Player_Name = []
Player_Price=[]
for row in driver.find_elements_by_xpath(".//div[@class='player']/a"):
    Player_Name.append(row.text)

for row in driver.find_elements_by_xpath(".//div[@class='rgt-col']/div[@class='rgt-hdr'][contains(.,'Salary')]/following-sibling::div"):
    Player_Price.append(row.text)

df = pd.DataFrame({"Player Name":Player_Name,"Salary":Player_Price})
print(df)

ModuleNotFoundError: No module named 'selenium'

In [15]:
 
class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df
    

def table_formatter(url):
    
    hp = HTMLTableParser()
    table = hp.parse_url(url)[0][1] # Grabbing the table from the tuple
    table.astype(str)
    
    return table
    
    
def main():
    #url = 'http://dnd5e.wikidot.com/wondrous-items'
    url = "https://www.aidedd.org/dnd-filters/magic-items.php#pied"
    #url = "https://donjon.bin.sh/5e/magic_items/"
    loot_table = table_formatter(url)
    pprint(loot_table)
    
    
    
main()


                                           Magic Item           Type   \
0                                     Adamantine Armor         Armor    
1                                          Alchemy Jug  Wondrous item   
2                             Ammunition +1, +2, or +3        Weapon    
3                                     Amulet of Health  Wondrous item   
4       Amulet of Proof against Detection and Location  Wondrous item   
..  ..                                             ...            ...   
271                                Well of Many Worlds  Wondrous item   
272                                              Whelm        Weapon    
273                                           Wind Fan  Wondrous item   
274                                       Winged Boots  Wondrous item   
275                                    Wings of Flying  Wondrous item   

                       Rarety  Attunement                        Source   
0                     uncommon              Dung